# Fichier générateur de modèles One Hot Encoding
## Bedbugs

In [23]:
import pandas as pd
import numpy as np
from math import sqrt, log
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pickle 

In [3]:
PATH = '.' #modifier si besoin le chemin du fichier
model_folder=PATH+"//models_sav/"

### Préparation et importation des données

In [5]:
# Lecture du fichier des données tests
#----------------------------------------------

# récupération du fichier contentant l'ensemble des données issues des requêtes
pricing_requests = pd.read_csv(PATH + '//pricing_requests_clean_addedFeatures.csv')
pricing_requests.head()

,hotel_id,price,stock,city,date,language,mobile,avatar_id,request_number,request_nb,group,brand,parking,pool,children_policy,pib,price_m2,nb_tourists,nb_hab_km2
0,105,144,102,amsterdam,39,austrian,1,6056,1,1.0,Yin Yang,Ardisson,0,0,0,990583.0,4610.0,7265.0,3530.0
1,880,113,81,amsterdam,39,austrian,1,6056,1,1.0,Boss Western,Boss Western,0,0,0,990583.0,4610.0,7265.0,3530.0
2,588,172,108,amsterdam,39,austrian,1,6056,1,1.0,Independant,Independant,0,0,0,990583.0,4610.0,7265.0,3530.0
3,352,105,84,amsterdam,39,austrian,1,6056,1,1.0,Independant,Independant,1,0,0,990583.0,4610.0,7265.0,3530.0
4,925,103,83,amsterdam,39,austrian,1,6056,1,1.0,Boss Western,Boss Western,1,0,0,990583.0,4610.0,7265.0,3530.0


In [6]:
# Assigner le bon type aux variables
#----------------------------------------------
#--- Convert to integer: 
int_list = ["date","avatar_id","hotel_id","stock","request_number","price", "request_nb", "pib", "price_m2", "nb_tourists", "nb_hab_km2"] 
pricing_requests[int_list] = pricing_requests[int_list].astype(int) 
#--- Convert to categorical: 
pricing_requests["city"] = pd.Categorical(pricing_requests["city"],ordered=False)
pricing_requests["language"] = pd.Categorical(pricing_requests["language"],ordered=False)
pricing_requests["mobile"] = pd.Categorical(pricing_requests["mobile"],ordered=False)
pricing_requests["parking"] = pd.Categorical(pricing_requests["parking"],ordered=False)
pricing_requests["pool"] = pd.Categorical(pricing_requests["pool"],ordered=False)
pricing_requests["children_policy"] = pd.Categorical(pricing_requests["children_policy"],ordered=False)
pricing_requests["group"] = pd.Categorical(pricing_requests["group"],ordered=False)
pricing_requests["brand"] = pd.Categorical(pricing_requests["brand"],ordered=False)

In [7]:
pricing_requests.dtypes

hotel_id              int32
price                 int32
stock                 int32
city               category
date                  int32
language           category
mobile             category
avatar_id             int32
request_number        int32
request_nb            int32
group              category
brand              category
parking            category
pool               category
children_policy    category
pib                   int32
price_m2              int32
nb_tourists           int32
nb_hab_km2            int32
dtype: object

In [11]:
# Variables à modifier pour rendre + gaussiennes 
pricing_requests["price"]=pricing_requests["price"].map(lambda x: sqrt(x))
pricing_requests["stock"]=pricing_requests["stock"].map(lambda x: log(x+1))

In [30]:
# Variables explicatives
#-------------------------------------------------
# Liste des variables categorical: 
cat_list1 = ["city", "language", "brand","mobile","parking","pool","children_policy"] #avec brand
cat_list2 = ["city", "language", "group","mobile","parking","pool","children_policy"] #avec group
cat_list3 = ["city", "language", "brand", "group","mobile","parking","pool","children_policy"] #avec brand & group
cat_list4 = ["city", "brand", "group","mobile","parking","pool","children_policy"]

# Liste des variables quantitatives: 
quant_list1 = ["hotel_id", "stock","request_nb", "date", "pib", "price_m2", "nb_tourists", "nb_hab_km2"] #on a enlevé avatar_id et price (=variable à expliquer)

# Création de la dataframe train
requests_pricingDum  = pd.get_dummies(pricing_requests[cat_list4],drop_first = True)
request_pricingQuant = pricing_requests[quant_list1]

dfC = pd.concat([requests_pricingDum,request_pricingQuant],axis=1)
dfC.columns

Index(['city_copenhagen', 'city_madrid', 'city_paris', 'city_rome',
       'city_sofia', 'city_valletta', 'city_vienna', 'city_vilnius',
       'brand_Ardisson', 'brand_Boss Western', 'brand_Chill Garden Inn',
       'brand_Corlton', 'brand_CourtYord', 'brand_Ibas', 'brand_Independant',
       'brand_J.Halliday Inn', 'brand_Marcure', 'brand_Morriot',
       'brand_Navatel', 'brand_Quadrupletree', 'brand_Royal Lotus',
       'brand_Safitel', 'brand_Tripletree', 'group_Boss Western',
       'group_Chillton Worldwide', 'group_Independant',
       'group_Morriott International', 'group_Yin Yang', 'mobile_1',
       'parking_1', 'pool_1', 'children_policy_1', 'children_policy_2',
       'hotel_id', 'stock', 'request_nb', 'date', 'pib', 'price_m2',
       'nb_tourists', 'nb_hab_km2'],
      dtype='object')

In [31]:
dfC

,city_copenhagen,city_madrid,city_paris,city_rome,city_sofia,city_valletta,city_vienna,city_vilnius,brand_Ardisson,brand_Boss Western,...,children_policy_1,children_policy_2,hotel_id,stock,request_nb,date,pib,price_m2,nb_tourists,nb_hab_km2
0,0,0,0,0,0,0,0,0,1,0,...,0,0,105,4.634729,1,39,990583,4610,7265,3530
1,0,0,0,0,0,0,0,0,0,1,...,0,0,880,4.406719,1,39,990583,4610,7265,3530
2,0,0,0,0,0,0,0,0,0,0,...,0,0,588,4.691348,1,39,990583,4610,7265,3530
3,0,0,0,0,0,0,0,0,0,0,...,0,0,352,4.442651,1,39,990583,4610,7265,3530
4,0,0,0,0,0,0,0,0,0,1,...,0,0,925,4.430817,1,39,990583,4610,7265,3530
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536083,0,0,0,0,1,0,0,0,0,0,...,0,0,190,0.000000,1,5,85008,1095,4973,7354
536084,0,0,0,0,1,0,0,0,0,0,...,0,0,107,0.000000,1,5,85008,1095,4973,7354
536085,0,0,0,0,1,0,0,0,0,0,...,0,0,883,1.609438,1,5,85008,1095,4973,7354
536086,0,0,0,0,1,0,0,0,0,0,...,0,0,353,2.639057,1,5,85008,1095,4973,7354


### Jeux de tests-trains

In [ ]:
# Variable à expliquer 
#-------------------------------------------------
Y = pricing_requests["price"]

In [33]:
# Construction de l'échantillon de test, échantillon de train et échantillon de validation
#-------------------------------------------------
X_train, X_test, Y_train, Y_test = train_test_split(dfC, Y, test_size=0.2, random_state=11)
X_train, X_val, Y_train, Y_val =  train_test_split(X_train, Y_train, test_size=0.25, random_state=11) # 0.25 x 0.8 = 0.2

In [34]:
print("X_train ---> taille: {} x {}".format(X_train.shape[0], X_train.shape[1]))
print("X_val   ---> taille: {}  x {}".format(X_val.shape[0], X_val.shape[1]))
print("X_test  ---> taille: {}  x {}".format(X_test.shape[0], X_test.shape[1]))

X_train ---> taille: 321652 x 41
X_val   ---> taille: 107218  x 41
X_test  ---> taille: 107218  x 41


In [35]:
# Standardisation des variables quantitatives 
#-------------------------------------------------
# standardisation
scaler = StandardScaler()

scaler.fit(X_train)
Xr_train = scaler.transform(X_train)
Xr_val = scaler.transform(X_val)
Xr_test = scaler.transform(X_test)

# transformation de Xr_train et Xr_test (numpy array) en dataframe
Xr_train = pd.DataFrame(Xr_train, index=X_train.index, columns=X_train.columns)
Xr_test = pd.DataFrame(Xr_test, index=X_test.index, columns=X_test.columns)
Xr_val = pd.DataFrame(Xr_val, index=X_val.index, columns=X_val.columns)

filename = model_folder + '12_11_One_hot_encoding_Standard_Scaler_Data2.sav'
pickle.dump(scaler, open(filename, 'wb'))

### Modèle(s)

In [ ]:
# create an xgboost regression model
import xgboost as xgb
#on utilise XGboost car il est considéré comme plus précis et plus fiable queGBM (de plus il utilise la prallélisation)

#on définit le modèle
model = xgb.XGBRegressor(n_estimators=2000, max_depth=10, min_child_weight=4, learning_rate=0.05, subsample=0.9, colsample_bytree=0.7, nthread=4, objective = 'reg:squarederror', silent=0)

#800
#on entraine le modèle 
model.fit(Xr_train,Y_train) #Xr = données normalisée

In [26]:
#Light XG boost : moins lourd
'''
import lightgbm as lgb
model = lgb.LGBMRegressor(colsample_bytree= 0.7,
 learning_rate= 0.5,
 max_depth= 10,
 n_estimators= 2000,
 nthread= 4,
 subsample=0.9,
 n_jobs = 2)
model.fit(Xr_train, Y_train, eval_set=[(Xr_test, Y_test),(Xr_train, Y_train)])'''

[LightGBM] [Warning] num_threads is set with n_jobs=2, nthread=4 will be ignored. Current value: num_threads=2
[1]	training's l2: 4.15737	valid_0's l2: 4.18222
[2]	training's l2: 2.16457	valid_0's l2: 2.18216
[3]	training's l2: 1.63137	valid_0's l2: 1.64065
[4]	training's l2: 1.20448	valid_0's l2: 1.21375
[5]	training's l2: 0.933929	valid_0's l2: 0.941984
[6]	training's l2: 0.828347	valid_0's l2: 0.835658
[7]	training's l2: 0.743904	valid_0's l2: 0.750989
[8]	training's l2: 0.660797	valid_0's l2: 0.665558
[9]	training's l2: 0.627402	valid_0's l2: 0.631742
[10]	training's l2: 0.600965	valid_0's l2: 0.605253
[11]	training's l2: 0.574031	valid_0's l2: 0.578471
[12]	training's l2: 0.546426	valid_0's l2: 0.549561
[13]	training's l2: 0.53198	valid_0's l2: 0.535178
[14]	training's l2: 0.517506	valid_0's l2: 0.520635
[15]	training's l2: 0.503867	valid_0's l2: 0.506741
[16]	training's l2: 0.486672	valid_0's l2: 0.490471
[17]	training's l2: 0.460966	valid_0's l2: 0.465143
[18]	training's l2: 0.4

[155]	training's l2: 0.0779103	valid_0's l2: 0.0821791
[156]	training's l2: 0.0770313	valid_0's l2: 0.0813408
[157]	training's l2: 0.0766461	valid_0's l2: 0.0809277
[158]	training's l2: 0.0760893	valid_0's l2: 0.0804134
[159]	training's l2: 0.0758962	valid_0's l2: 0.0802181
[160]	training's l2: 0.0757077	valid_0's l2: 0.0800375
[161]	training's l2: 0.0752856	valid_0's l2: 0.0796482
[162]	training's l2: 0.0748678	valid_0's l2: 0.0791835
[163]	training's l2: 0.0746013	valid_0's l2: 0.0789195
[164]	training's l2: 0.0743653	valid_0's l2: 0.0786843
[165]	training's l2: 0.0737363	valid_0's l2: 0.0780169
[166]	training's l2: 0.0734314	valid_0's l2: 0.0777501
[167]	training's l2: 0.0732261	valid_0's l2: 0.0775831
[168]	training's l2: 0.073079	valid_0's l2: 0.0774518
[169]	training's l2: 0.0727517	valid_0's l2: 0.0770965
[170]	training's l2: 0.0725068	valid_0's l2: 0.076842
[171]	training's l2: 0.072389	valid_0's l2: 0.0767396
[172]	training's l2: 0.0718856	valid_0's l2: 0.076218
[173]	training

[313]	training's l2: 0.0420275	valid_0's l2: 0.0460803
[314]	training's l2: 0.0418307	valid_0's l2: 0.0458613
[315]	training's l2: 0.0415224	valid_0's l2: 0.0455314
[316]	training's l2: 0.0412834	valid_0's l2: 0.0453078
[317]	training's l2: 0.0411129	valid_0's l2: 0.0451488
[318]	training's l2: 0.0409442	valid_0's l2: 0.0449312
[319]	training's l2: 0.0406193	valid_0's l2: 0.0445778
[320]	training's l2: 0.0404989	valid_0's l2: 0.0444729
[321]	training's l2: 0.040459	valid_0's l2: 0.0444352
[322]	training's l2: 0.0402986	valid_0's l2: 0.0442593
[323]	training's l2: 0.0402717	valid_0's l2: 0.0442563
[324]	training's l2: 0.040255	valid_0's l2: 0.044244
[325]	training's l2: 0.0402326	valid_0's l2: 0.0442367
[326]	training's l2: 0.04013	valid_0's l2: 0.0441445
[327]	training's l2: 0.0400542	valid_0's l2: 0.0440668
[328]	training's l2: 0.040039	valid_0's l2: 0.0440626
[329]	training's l2: 0.0400173	valid_0's l2: 0.0440465
[330]	training's l2: 0.0399458	valid_0's l2: 0.0439817
[331]	training's

[467]	training's l2: 0.0287588	valid_0's l2: 0.0328984
[468]	training's l2: 0.0286608	valid_0's l2: 0.0327808
[469]	training's l2: 0.0285776	valid_0's l2: 0.0327099
[470]	training's l2: 0.0284747	valid_0's l2: 0.032599
[471]	training's l2: 0.0284091	valid_0's l2: 0.0325274
[472]	training's l2: 0.0282516	valid_0's l2: 0.0323507
[473]	training's l2: 0.0281963	valid_0's l2: 0.032307
[474]	training's l2: 0.0281844	valid_0's l2: 0.0322985
[475]	training's l2: 0.0281049	valid_0's l2: 0.0322133
[476]	training's l2: 0.0280014	valid_0's l2: 0.0321122
[477]	training's l2: 0.0278949	valid_0's l2: 0.0320166
[478]	training's l2: 0.0278846	valid_0's l2: 0.0320171
[479]	training's l2: 0.0278731	valid_0's l2: 0.0320154
[480]	training's l2: 0.0278538	valid_0's l2: 0.0319979
[481]	training's l2: 0.0277996	valid_0's l2: 0.0319482
[482]	training's l2: 0.0277866	valid_0's l2: 0.03194
[483]	training's l2: 0.0277539	valid_0's l2: 0.0319105
[484]	training's l2: 0.0277387	valid_0's l2: 0.0319013
[485]	training

[624]	training's l2: 0.0217075	valid_0's l2: 0.0260154
[625]	training's l2: 0.021701	valid_0's l2: 0.026014
[626]	training's l2: 0.0216418	valid_0's l2: 0.0259418
[627]	training's l2: 0.021623	valid_0's l2: 0.0259335
[628]	training's l2: 0.0216169	valid_0's l2: 0.0259339
[629]	training's l2: 0.0215805	valid_0's l2: 0.0258737
[630]	training's l2: 0.0215691	valid_0's l2: 0.0258661
[631]	training's l2: 0.0215585	valid_0's l2: 0.0258645
[632]	training's l2: 0.02154	valid_0's l2: 0.0258509
[633]	training's l2: 0.0215298	valid_0's l2: 0.0258513
[634]	training's l2: 0.0215218	valid_0's l2: 0.025846
[635]	training's l2: 0.0215127	valid_0's l2: 0.0258424
[636]	training's l2: 0.0215035	valid_0's l2: 0.0258422
[637]	training's l2: 0.0214922	valid_0's l2: 0.0258409
[638]	training's l2: 0.0214645	valid_0's l2: 0.0258147
[639]	training's l2: 0.021457	valid_0's l2: 0.0258166
[640]	training's l2: 0.0214515	valid_0's l2: 0.0258178
[641]	training's l2: 0.0214434	valid_0's l2: 0.0258114
[642]	training's 

...

[1532]	training's l2: 0.00920064	valid_0's l2: 0.0139632
[1533]	training's l2: 0.00918538	valid_0's l2: 0.0139435
[1534]	training's l2: 0.00916787	valid_0's l2: 0.0139271
[1535]	training's l2: 0.00915667	valid_0's l2: 0.0139136
[1536]	training's l2: 0.00915126	valid_0's l2: 0.0139081
[1537]	training's l2: 0.00912787	valid_0's l2: 0.0138748
[1538]	training's l2: 0.00911777	valid_0's l2: 0.0138604
[1539]	training's l2: 0.00911517	valid_0's l2: 0.0138612
[1540]	training's l2: 0.00910634	valid_0's l2: 0.0138508
[1541]	training's l2: 0.00908652	valid_0's l2: 0.0138264
[1542]	training's l2: 0.00907227	valid_0's l2: 0.0138123
[1543]	training's l2: 0.00906375	valid_0's l2: 0.0138036
[1544]	training's l2: 0.00906124	valid_0's l2: 0.0138018
[1545]	training's l2: 0.00905925	valid_0's l2: 0.0138012
[1546]	training's l2: 0.00905618	valid_0's l2: 0.0137981
[1547]	training's l2: 0.00905441	valid_0's l2: 0.0137975
[1548]	training's l2: 0.00905199	valid_0's l2: 0.0137964
[1549]	training's l2: 0.0090503

[1679]	training's l2: 0.00845198	valid_0's l2: 0.0132815
[1680]	training's l2: 0.00845048	valid_0's l2: 0.0132807
[1681]	training's l2: 0.00844387	valid_0's l2: 0.0132748
[1682]	training's l2: 0.00844255	valid_0's l2: 0.0132746
[1683]	training's l2: 0.0084403	valid_0's l2: 0.0132752
[1684]	training's l2: 0.00843593	valid_0's l2: 0.0132725
[1685]	training's l2: 0.00843446	valid_0's l2: 0.0132731
[1686]	training's l2: 0.00843334	valid_0's l2: 0.0132722
[1687]	training's l2: 0.00841195	valid_0's l2: 0.0132407
[1688]	training's l2: 0.00841018	valid_0's l2: 0.0132383
[1689]	training's l2: 0.00838711	valid_0's l2: 0.013208
[1690]	training's l2: 0.00836773	valid_0's l2: 0.0131808
[1691]	training's l2: 0.0083612	valid_0's l2: 0.0131728
[1692]	training's l2: 0.00833987	valid_0's l2: 0.013145
[1693]	training's l2: 0.00830827	valid_0's l2: 0.0131156
[1694]	training's l2: 0.0083054	valid_0's l2: 0.0131137
[1695]	training's l2: 0.00829613	valid_0's l2: 0.0131051
[1696]	training's l2: 0.00828343	val

[1825]	training's l2: 0.00758446	valid_0's l2: 0.0123633
[1826]	training's l2: 0.00757411	valid_0's l2: 0.0123495
[1827]	training's l2: 0.00756719	valid_0's l2: 0.0123435
[1828]	training's l2: 0.00756008	valid_0's l2: 0.0123346
[1829]	training's l2: 0.00755035	valid_0's l2: 0.0123185
[1830]	training's l2: 0.00754904	valid_0's l2: 0.012318
[1831]	training's l2: 0.0075405	valid_0's l2: 0.0123069
[1832]	training's l2: 0.00752351	valid_0's l2: 0.0122839
[1833]	training's l2: 0.00751475	valid_0's l2: 0.0122761
[1834]	training's l2: 0.00751299	valid_0's l2: 0.0122749
[1835]	training's l2: 0.00751097	valid_0's l2: 0.0122746
[1836]	training's l2: 0.00750134	valid_0's l2: 0.0122624
[1837]	training's l2: 0.00749616	valid_0's l2: 0.0122563
[1838]	training's l2: 0.00749168	valid_0's l2: 0.012251
[1839]	training's l2: 0.00748854	valid_0's l2: 0.0122458
[1840]	training's l2: 0.00747741	valid_0's l2: 0.0122298
[1841]	training's l2: 0.00746903	valid_0's l2: 0.0122212
[1842]	training's l2: 0.00746668	v

[1974]	training's l2: 0.00691181	valid_0's l2: 0.0116723
[1975]	training's l2: 0.00691033	valid_0's l2: 0.0116722
[1976]	training's l2: 0.0069082	valid_0's l2: 0.0116722
[1977]	training's l2: 0.00690645	valid_0's l2: 0.011671
[1978]	training's l2: 0.00689974	valid_0's l2: 0.0116649
[1979]	training's l2: 0.0068954	valid_0's l2: 0.0116599
[1980]	training's l2: 0.00689366	valid_0's l2: 0.0116581
[1981]	training's l2: 0.00689264	valid_0's l2: 0.0116581
[1982]	training's l2: 0.00688347	valid_0's l2: 0.0116409
[1983]	training's l2: 0.00687678	valid_0's l2: 0.0116307
[1984]	training's l2: 0.00686667	valid_0's l2: 0.0116161
[1985]	training's l2: 0.00685891	valid_0's l2: 0.0116067
[1986]	training's l2: 0.00685793	valid_0's l2: 0.0116066
[1987]	training's l2: 0.00684685	valid_0's l2: 0.0115895
[1988]	training's l2: 0.0068364	valid_0's l2: 0.0115743
[1989]	training's l2: 0.00682687	valid_0's l2: 0.0115563
[1990]	training's l2: 0.00682125	valid_0's l2: 0.0115494
[1991]	training's l2: 0.00681443	va

LGBMRegressor(colsample_bytree=0.7, learning_rate=0.5, max_depth=10,
              n_estimators=2000, n_jobs=2, nthread=4, subsample=0.9)

In [ ]:
from sklearn.metrics import mean_squared_error
ypred = model.predict(Xr_test) #on prédit sur Xr_test
mse = mean_squared_error(Y_test**2, ypred**2) 
#Calcul du mse entre les prédictions. ATTENTION : les mettre au carré 
#car le prix doit être au carré car on a fait une transformation racine carrée
print("MSE: %.2f" % mse)
print(model.score(Xr_test, Y_test))# score

In [ ]:
#save le modèle :
tune_hyperparamters = True
if tune_hyperparamters :
    filename = model_folder + '12-11_One_hot_encoding_Xgboost2.sav' # enregistrement du modèle dans X_gboost
    pickle.dump(model, open(filename, 'wb')) # enregistrement du fichier 